In [ ]:
import pickle
import json
import re
import nltk
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors


nltk.download('punkt')
nltk.download('stopwords')

pd.set_option('display.max_columns', None)

In [125]:
df = pd.read_csv('https://raw.githubusercontent.com/Ichi-1/AnimeDb-Backend-DRF/main/docs/db_for_reccomendation_sys.csv')
df.head(1)

,anime_id,age_rating,age_rating_guide,average_rating,description,episode_count,episode_length,kind,poster_image,season,studio,tags,title,title_jp,total_length,year,year_end
0,1,R,17+ (violence & profanity),82.47,"In the year 2071, humanity has colonized sever...",26,25,TV,https://media.kitsu.io/anime/poster_images/1/o...,Spring,Sunrise,"Action, Adventure, Drama, Sci Fi, Bounty Hunte...",Cowboy Bebop,カウボーイビバップ,650,1998.0,1999.0


### Data Preprocessing

In [126]:
# remove numbers, punctuation, spaces, etc.
df['clean_description'] = df['description'].str.lower()
df['clean_description'] = df['clean_description'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))
df['clean_description'] = df['clean_description'].apply(lambda x: re.sub('\s+', ' ', x))

# tokenize sentence
df['clean_description'] = df['clean_description'].apply(lambda x: nltk.word_tokenize(x))

# remove stop words 
stop_words = nltk.corpus.stopwords.words('english')

In [127]:
description = []

for sentence in df['clean_description']:
  temp = []
  for word in sentence:
    if word not in stop_words and len(word) > 3:
      temp.append(word)
  description.append(temp)


In [128]:
df['clean_description'] = description

In [129]:
df['tags'] = df['tags'].apply(lambda x: str(x).split(','))
df['studio'] = df['studio'].apply(lambda x: str(x).split(','))
df['kind'] = df['kind'].apply(lambda x: x.split(','))

In [130]:
def clean(list):
  temp = []
  for word in list:
    temp.append(word.lower().replace(' ', ''))
  return temp


df['tags'] = [clean(x) for x in df['tags']]
df['studio'] = [clean(x) for x in df['studio']]
df['kind'] = [clean(x) for x in df['kind']]


df['kind'][555]


['special']

In [ ]:
# combining all the columns data
columns = ['clean_description', 'tags', 'studio', 'kind']
clean_input = []

for i in range(df.shape[0]):
  words = ''
  for col in columns:
    words += ' '.join(df[col][i]) + ' '
  clean_input.append(words)

clean_input[0]

### Clean Dataframe

In [240]:
import csv

clean_df = pd.read_csv('https://raw.githubusercontent.com/Ichi-1/AnimeDb-Backend-DRF/main/docs/db_for_reccomendation_sys.csv')
clean_df['clean_input'] = clean_input
clean_df['year'] = clean_df['year'].astype('Int64')



clean_df = clean_df[['anime_id', 'title', 'poster_image', 'kind', 'year']]

clean_df.head(3)

# index df conatain only 'title' column
index = pd.Series(clean_df['title'])

,anime_id,title,poster_image,kind,year
0,1,Cowboy Bebop,https://media.kitsu.io/anime/poster_images/1/o...,TV,1998
1,2,Cowboy Bebop: The Movie,https://media.kitsu.io/anime/poster_images/2/o...,movie,2001
2,3,Trigun,https://media.kitsu.io/anime/poster_images/3/o...,TV,1998


In [208]:
def find_recommendation_indxs(title:str) -> list:
  idx = index[index == title].index[0]
  score = pd.Series(cs_matrix[idx]).sort_values(ascending=False)
  top_50 = list(score.iloc[1:10].index)

  recommendations = clean_df[['anime_id','title', 'poster_image', 'kind', 'year']]\
    .iloc[top_50].to_json(orient='records')

  return recommendations

In [ ]:
res = find_recommendation_indxs('Death Note')

json.loads(res)